In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
one_hot_encoder_geo = OneHotEncoder()
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [7]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [8]:
X = data.drop('Exited',axis=1)
y = data['Exited']

In [9]:
X_train,X_test,y_train, y_test= train_test_split(X,y,test_size = 0.2,random_state=42)

#Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Save the Encoded File as pickle
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file) 

In [10]:
## Define a function to create the model and try different parameters
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [11]:
#Create a Keras Classifier, this keras classifier is responsible for creating an entire neural network or ANN

model =KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,verbose=0)

In [12]:
#Define the GridSearch
param_grid = {
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}
import tensorflow as tf
print("GPUs Available:", tf.config.list_physical_devices('GPU'))
tf.debugging.set_log_device_placement(True)


GPUs Available: []


In [13]:
#Perform Grid Search
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,verbose=1,cv=3)
grid_result = grid.fit(X_train,y_train)

#print the best parameter
print("Best:  %f using %s" % (grid_result.best_score_,grid_result.best_params_))

AttributeError: 'super' object has no attribute '__sklearn_tags__'